<a href="https://colab.research.google.com/github/Uttam1998/tshirt_type_detection/blob/main/Tshirt_type_detection(vgg19).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/data/train'
valid_path = '/content/drive/MyDrive/data/test'

In [5]:
from tensorflow.keras.applications.vgg19 import VGG19

In [7]:
vg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights="imagenet", include_top=False)

80142336/80134624 [==============================] - 0s 0us/step


In [8]:
for layer in vg19.layers:
    layer.trainable = False

In [9]:
x = Flatten()(vg19.output)

In [11]:
# useful for getting number of output classes
folders = glob('/content/drive/MyDrive/data/train/*')

In [12]:
prediction = Dense(len(folders), activation="softmax")(x)

modelvg = Model(inputs=vg19.input, outputs=prediction)

In [13]:
modelvg.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [14]:
modelvg.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [15]:
train_datagen_vg19 = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen_vg19 = ImageDataGenerator(rescale=1./255)

In [16]:
trainning_set_vg19 = train_datagen_vg19.flow_from_directory(train_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode="categorical")

Found 2875 images belonging to 4 classes.


In [18]:
testing_set_vg19 = test_datagen_vg19.flow_from_directory(valid_path,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode="categorical")

Found 500 images belonging to 4 classes.


In [19]:
r_vg19 = modelvg.fit_generator(trainning_set_vg19,
                       validation_data=testing_set_vg19,
                       epochs=10,
                       steps_per_epoch=len(trainning_set_vg19),
                       validation_steps=len(testing_set_vg19)
                       )

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
90/90 [==============================] - 807s 9s/step - loss: 1.4312 - accuracy: 0.5241 - val_loss: 0.6290 - val_accuracy: 0.7620
Epoch 2/10
90/90 [==============================] - 37s 413ms/step - loss: 0.4963 - accuracy: 0.8169 - val_loss: 0.5160 - val_accuracy: 0.8020
Epoch 3/10
90/90 [==============================] - 37s 413ms/step - loss: 0.4405 - accuracy: 0.8392 - val_loss: 0.4669 - val_accuracy: 0.8240
Epoch 4/10
90/90 [==============================] - 37s 413ms/step - loss: 0.3617 - accuracy: 0.8739 - val_loss: 0.5950 - val_accuracy: 0.7600
Epoch 5/10
90/90 [==============================] - 37s 415ms/step - loss: 0.2759 - accuracy: 0.9030 - val_loss: 0.6103 - val_accuracy: 0.8040
Epoch 6/10
90/90 [==============================] - 37s 416ms/step - loss: 0.2516 - accuracy: 0.9148 - val_loss: 0.7379 - val_accuracy: 0.7760
Epoch 7/10
90/90 [==============================] - 37s 413ms/step - loss: 0.2439 - accuracy: 0.9037 - val_loss: 0.4376 - val_accuracy: 0.8400
E

In [20]:
y_pred = modelvg.predict(testing_set_vg19)
y_pred

array([[9.9165654e-01, 7.5576075e-08, 4.0448532e-03, 4.2984756e-03],
       [2.7664660e-02, 4.6528065e-01, 1.0271713e-01, 4.0433753e-01],
       [6.9278038e-05, 9.8215733e-06, 8.7126801e-03, 9.9120820e-01],
       ...,
       [1.0729419e-05, 1.8634013e-05, 7.2472083e-04, 9.9924588e-01],
       [2.7421381e-02, 9.3444665e-05, 6.6289879e-02, 9.0619540e-01],
       [1.2520791e-03, 9.4124806e-01, 4.8474218e-03, 5.2652355e-02]],
      dtype=float32)

In [21]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([0, 1, 3, 2, 2, 1, 2, 0, 3, 3, 2, 3, 3, 3, 1, 2, 1, 0, 2, 3, 1, 0,
       3, 2, 3, 3, 3, 1, 3, 0, 3, 1, 3, 0, 0, 3, 2, 1, 3, 2, 3, 1, 0, 3,
       3, 2, 3, 0, 2, 2, 2, 3, 3, 3, 2, 3, 0, 2, 2, 2, 3, 0, 3, 3, 0, 1,
       2, 3, 3, 2, 2, 0, 3, 1, 1, 3, 1, 3, 1, 0, 3, 3, 3, 2, 3, 1, 3, 3,
       0, 3, 2, 2, 1, 1, 2, 1, 0, 2, 3, 3, 1, 0, 3, 3, 3, 0, 1, 0, 3, 0,
       3, 3, 0, 0, 2, 0, 3, 3, 0, 1, 2, 1, 3, 2, 2, 1, 1, 3, 0, 3, 2, 0,
       1, 3, 1, 0, 2, 1, 3, 1, 3, 2, 3, 3, 0, 3, 0, 0, 1, 3, 1, 3, 0, 3,
       3, 2, 2, 3, 3, 3, 1, 3, 1, 1, 3, 1, 0, 3, 0, 0, 3, 0, 0, 2, 1, 2,
       1, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 2, 3, 3, 2, 1, 3, 3, 3, 2, 3,
       3, 0, 2, 0, 3, 2, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 2, 3, 1, 0,
       3, 1, 2, 1, 3, 2, 3, 0, 3, 0, 3, 3, 0, 0, 3, 2, 0, 3, 3, 3, 3, 1,
       1, 2, 3, 3, 2, 0, 1, 3, 1, 1, 2, 3, 3, 1, 3, 3, 2, 1, 3, 0, 3, 0,
       0, 2, 2, 3, 0, 0, 2, 2, 1, 0, 2, 3, 1, 1, 0, 1, 3, 3, 3, 2, 1, 3,
       3, 2, 3, 3, 1, 0, 3, 1, 3, 1, 3, 2, 3, 0, 1,

In [24]:
class_dict = trainning_set_vg19.class_indices
class_dict

{'Full_Sleeves': 0, 'Polo_Tshirt': 1, 'round': 2, 'sleeveless': 3}

In [22]:
modelvg.save("vgg_19tl.model")

INFO:tensorflow:Assets written to: vgg_19tl.model/assets


In [23]:
import cv2
import tensorflow as tf
def prepare(filepath):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    new_array = cv2.resize(img_array, (224, 224))
    return new_array.reshape(-1, 224, 224, 3)

model = tf.keras.models.load_model("vgg_19tl.model")

In [28]:
prediction = model.predict([prepare("/content/polo1.jpg")])
np.argmax(prediction)

1

In [29]:
if np.argmax(prediction) == 0:
    print("Full_Sleeves")
elif np.argmax(prediction) == 1:
    print("Polo_Tshirt")
elif np.argmax(prediction) == 2:
    print("round")
else:
    print("sleeveless")

Polo_Tshirt
